### Installation
For linux:
- install nupack and add 'export NUPACKHOME=/path/to/nupack3.2.1' to ~/.bashrc
- test that Nupack is in path with "echo $NUPACKHOME"
- run a Python 2.7 environment with the following additional packages installed:
 - matplotlib 2.2.5
 - Bio 1.76


# Orthogonal sequence optimizer
## USER INPUT:
- add sequences to be optimized as a dictionary of strings
- add sequences to be fixed (and avoided during optimization) as a dictionary of strings
- set the global physical parameters temperature, sodium concentration (mM), and magnesium concentration (mM)

In [10]:
import imp
from sequence_analysis_lib import sequence_analysis_lib as sq
import pynupack as nu

In [11]:
inp = {
    "p_red" : [sq.random_seq_of_length(20), 60],#"GACTGGCTATTCTGACATAC",
    "GR_OE" : [sq.random_seq_of_length(20), 60],#"CCTCCATTGTGACGTACCTT",
    "GR_overlap" : [sq.random_seq_of_length(20), 60],#"TATACTCAGCATCATATGCG",
    "Tar_overlap" : [sq.random_seq_of_length(20), 60],#"ACTGCTAGCTAGGTTCAGTA",
    "p_yel" : [sq.random_seq_of_length(20),60],#"ACCTACTTCAATCTTCAACG",
    "BY_OE" : [sq.random_seq_of_length(20),60],#"GCCTAAATCTAGTTATGCCC",
    "p_tar" : [sq.random_seq_of_length(20),60]#"CGCTAGTTAGTGTGTAGCCA",
    
}

fix = {
    "illumina1" : "CAAGCAGAAGAC",
    "illumina2" : "CAAGTTGTCA",
    "illumina3" : "GGCATACGAGAT",
    "illumina4" : "GTCTCGTGGGCTCGGAG",
    "illumina5" : "ATGTGTATAAGAGACAG",
    "illumina6" : "GTGTAGATCTCGG",
    "illumina7" : "TCATTGCACG",
    "illumina8" : "TGGTCGCCGTATCATT",
    "illumina9" : "CTGTCTCTTATACA",
   "illumina10" : "CATCTGACGCTGCCGACGA"    
}
xtr = {
        "bc_r1" : "NNNNNNNNNN",#"ACCTCCACCT",
        "bc_ri" : "CCACC",
        "bc_r2" : "NNNNNNNNNN",#"TCACTACTCA",
    
        "GR_evL" : "NNNNN",#"CACTA",
        "GR_evT" : "NNNNN",#"ACCTA",
    
        "bc_y1" : "NNNNNNNNNN",#"CACCTCACCT",
        "bc_yi" : "CCACT",
        "bc_y2" : "NNNNNNNNNN",#"ACTCCCACTC",
    
        "BY_evL" : "NNNNN",#"TACTT",
        "BY_evT" : "NNNNN",#"TACTA",
               
    "bc_tar" : "NNNNNNNNNNNNNNNNNNNN",#"TGTTCGTGTCTTGTTCTTGT"
}


global T, magnesium, sodium,tm_target
T = 60
magnesium = .2 
sodium = 1 # all monovalent Na+ and K+ and tris+
# dNTPs = .1
# percentDMSO = 5 #% has the effect of -.75degreesC per percentage dmso


In [ ]:
imp.reload(sq)


### run the optimizer with small chance of reversion to escape local minima ###
### this means that sometimes a change that worsens the score will occur    ###
new_domains_coarse = sq.optimize_domains_quick(inp,
 fix,
 iterations=100, 
 algorithm_reversion_probability= 0.1, 
 plot_interval=100,
 file_prefix = "coarse",
 T = T, magnesium = magnesium, sodium=sodium)

### run the optimizer without reversion - only improvements accepted ###
inp = sq.optimize_domains_quick(new_domains_coarse,
 fix,
 iterations=100,                              
 algorithm_reversion_probability= 0.0, 
 plot_interval=100,
 file_prefix = "fine",
 T = T, magnesium = magnesium, sodium=sodium)

strands_dict = {
"GR_amp_s" : inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]  ,
"GR_OEL_s" : sq.prime(inp["GR_overlap"][0])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"][0]),
"GR_OET_s" : inp["Tar_overlap"][0]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"][0]),
"GR_monL_s" : inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evL"]+inp["GR_overlap"][0],
"GR_monT_s" : inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"][0]),
"BY_amp_s" : inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0],
"BY_OEL_s" :inp["GR_overlap"][0]+sq.prime(xtr["BY_evL"])+sq.prime(inp["BY_OE"][0]),
"BY_OET_s" : inp["Tar_overlap"][0]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"][0]),
"BY_monL_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"][0]),
"BY_monT_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"][0]),
"Tar_mon_s" :  inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0],
"full_loc_r_s" :  inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evL"]+inp["GR_overlap"][0]+sq.prime(xtr["BY_evL"])+ sq.prime(inp["BY_OE"][0])+ sq.prime(xtr["bc_y2"])+ sq.prime(xtr["bc_yi"])+ sq.prime(xtr["bc_y1"])+ sq.prime(inp["p_yel"][0]),
"full_loc_y_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"][0])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"][0])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"][0]),
"full_tar_r_s" :inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"][0])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"][0]),
"full_tar_y_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"][0])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"][0]),
"full_tar_tarR_s" :inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"][0])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"][0]),
"full_tar_tarY_s" :inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"][0])+sq.prime(xtr["bc_y2"])+sq.prime(xtr["bc_yi"])+sq.prime(xtr["bc_y1"])+sq.prime(inp["p_yel"][0]),
    }

strands_order_dict = {
"p_red" : inp["p_red"][0],
"p_yel" : inp["p_yel"][0],
"GR_amp_s" : sq.prime(inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0])  ,
"GR_OEL_s" : sq.prime(inp["GR_overlap"][0])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"][0]),
"GR_OET_s" : inp["Tar_overlap"][0]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"][0]),
# "GR_monL_s" : inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evL"]+inp["GR_overlap"][0],
# "GR_monT_s" : inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"][0]),
"BY_amp_s" : sq.prime(inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]),
"BY_OEL_s" :inp["GR_overlap"][0]+sq.prime(xtr["BY_evL"])+sq.prime(inp["BY_OE"][0]),
"BY_OET_s" : inp["Tar_overlap"][0]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"][0]),
# "BY_monL_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"][0]),
# "BY_monT_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"][0]),
# "Tar_mon_s" :  inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0],
# "full_loc_r_s" :  inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evL"]+inp["GR_overlap"][0]+sq.prime(xtr["BY_evL"])+ sq.prime(inp["BY_OE"][0])+ sq.prime(xtr["bc_y2"])+ sq.prime(xtr["bc_yi"])+ sq.prime(xtr["bc_y1"])+ sq.prime(inp["p_yel"][0]),
# "full_loc_y_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evL"]+sq.prime(inp["GR_overlap"][0])+sq.prime(xtr["GR_evL"])+sq.prime(inp["GR_OE"][0])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"][0]),
# "full_tar_r_s" :inp["p_red"][0]+xtr["bc_r1"]+xtr["bc_ri"]+xtr["bc_r2"]+inp["GR_OE"][0]+xtr["GR_evT"]+sq.prime(inp["Tar_overlap"][0])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"][0]),
# "full_tar_y_s" :inp["p_yel"][0]+xtr["bc_y1"]+xtr["bc_yi"]+xtr["bc_y2"]+inp["BY_OE"][0]+xtr["BY_evT"]+sq.prime(inp["Tar_overlap"][0])+sq.prime(xtr["bc_tar"])+sq.prime(inp["p_tar"][0]),
# "full_tar_tarR_s" :inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0]+sq.prime(xtr["GR_evT"])+sq.prime(inp["GR_OE"][0])+sq.prime(xtr["bc_r2"])+sq.prime(xtr["bc_ri"])+sq.prime(xtr["bc_r1"])+sq.prime(inp["p_red"][0]),
# "full_tar_tarY_s" :inp["p_tar"][0]+xtr["bc_tar"]+inp["Tar_overlap"][0]+sq.prime(xtr["BY_evT"])+sq.prime(inp["BY_OE"][0])+sq.prime(xtr["bc_y2"])+sq.prime(xtr["bc_yi"])+sq.prime(xtr["bc_y1"])+sq.prime(inp["p_yel"][0]),
    }

(0, 2191.464, 1123.4012000000002)
('new variable domains: ', ['GCCATGCAGTGTAGTAGGAC', 'AAAAGTTGCCATCGCCAGTA', 'TCAAGCGCAATACGGAAGGC', 'TATAAGTGATTAATTTGGTT', 'ACATAGTTCGGTAAGAGCAC', 'TCGTCGGAGTCAGGTGAACG', 'TCCTAAAAGACTGTGGCATT'])
(1, 2224.6313999999993, 4002.328)
(2, 1305.4663999999996, 1243.8446)
(3, 8232.6375, 11576.405199999997)
(4, 1112.2631999999999, 3022.111499999999)
(5, 2148.5989999999997, 1151.6534000000001)
(6, 4612.3769999999995, 1283.5083999999997)
(7, 1123.5524, 1161.6500000000003)
(8, 2219.865, 736.1686000000001)
(9, 1247.2531999999997, 2338.41)
(10, 8259.84, 9365.2455)
(11, 1112.2631999999999, 374.7966)
(12, 1152.287, 354.11800000000005)
(13, 1278.4306, 1389.9966)
(14, 1130.198, 1259.1103999999996)
(15, 734.7478000000001, 823.3839999999998)
(16, 1242.5497999999998, 399.1474)
(17, 8262.999000000002, 7500.1075)
(18, 374.1438, 1908.1064)
(19, 357.43240000000003, 1117.5620000000001)
(20, 1277.9392, 410.65659999999997)
(21, 1132.2644000000003, 351.9416000000001)
(22, 729.194

In [ ]:
### print out names and sequences of assembled strands defined as combinations of domains ###
print(strands_dict)

In [ ]:
### print out domain names and sequences after optimization ###
print(inp)

In [ ]:
### print out domain reverse complements ###
sq.get_complements_list([seq for seq,temp in inp.values()])

In [ ]:
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
### print out Wallace melting temps of the domains ###
for seq in inp.keys():
    tm_wallace = mt.Tm_Wallace(inp[seq])
    print(seq,  tm_wallace)

In [ ]:
strands_order_dict

In [ ]:
sq.get_complements_list(strands_order_dict.values())

In [ ]:
strands_order_dict.keys()

In [8]:
import svgwrite

dwg = svgwrite.Drawing('test.svg', profile='tiny')
dwg.add(dwg.text('Test', insert=(0, 0.2), fill=colors['Maroon']))
dwg.save()


In [5]:
colors = dict([
    ["alice blue", "#F0F8FF"],
    ["antique white", "#FAEBD7"],
    ["aqua", "#00FFFF"],
    ["aqua marine", "#7FFFD4"],
    ["azure", "#F0FFFF"],
    ["beige", "#F5F5DC"],
    ["bisque", "#FFE4C4"],
    ["Black", "#000000"],
    ["black", "#000000"],
    ["blanched almond", "#FFEBCD"],
    ["Blue", "#0000FF"],
    ["blue", "#0000FF"],
    ["blue violet", "#8A2BE2"],
    ["brown", "#A52A2A"],
    ["burly wood", "#DEB887"],
    ["cadet blue", "#5F9EA0"],
    ["chart reuse", "#7FFF00"],
    ["chocolate", "#D2691E"],
    ["coral", "#FF7F50"],
    ["corn flower blue", "#6495ED"],
    ["corn silk", "#FFF8DC"],
    ["crimson", "#DC143C"],
    ["cyan", "#00FFFF"],
    ["Cyan-Aqua", "#00FFFF"],
    ["dark blue", "#00008B"],
    ["dark cyan", "#008B8B"],
    ["dark golden rod", "#B8860B"],
    ["dark gray-dark grey", "#A9A9A9"],
    ["dark green", "#006400"],
    ["dark khaki", "#BDB76B"],
    ["dark magenta", "#8B008B"],
    ["dark olive green", "#556B2F"],
    ["dark orange", "#FF8C00"],
    ["dark orchid", "#9932CC"],
    ["dark red", "#8B0000"],
    ["dark red", "#8B0000"],
    ["dark salmon", "#E9967A"],
    ["dark sea green", "#8FBC8F"],
    ["dark slate blue", "#483D8B"],
    ["dark slate gray", "#2F4F4F"],
    ["dark turquoise", "#00CED1"],
    ["dark violet", "#9400D3"],
    ["deep pink", "#FF1493"],
    ["deep sky blue", "#00BFFF"],
    ["dim gray-dim grey", "#696969"],
    ["dodger blue", "#1E90FF"],
    ["firebrick", "#B22222"],
    ["floral white", "#FFFAF0"],
    ["forest green", "#228B22"],
    ["gainsboro", "#DCDCDC"],
    ["ghost white", "#F8F8FF"],
    ["gold", "#FFD700"],
    ["golden rod", "#DAA520"],
    ["Gray", "#808080"],
    ["gray-grey", "#808080"],
    ["Green", "#008000"],
    ["green", "#008000"],
    ["green yellow", "#ADFF2F"],
    ["honeydew", "#F0FFF0"],
    ["hot pink", "#FF69B4"],
    ["indian red", "#CD5C5C"],
    ["indigo", "#4B0082"],
    ["ivory", "#FFFFF0"],
    ["khaki", "#F0E68C"],
    ["lavender", "#E6E6FA"],
    ["lavender blush", "#FFF0F5"],
    ["lawn green", "#7CFC00"],
    ["lemon chiffon", "#FFFACD"],
    ["light blue", "#ADD8E6"],
    ["light coral", "#F08080"],
    ["light cyan", "#E0FFFF"],
    ["light golden rod yellow", "#FAFAD2"],
    ["light gray-light grey", "#D3D3D3"],
    ["light green", "#90EE90"],
    ["light pink", "#FFB6C1"],
    ["light salmon", "#FFA07A"],
    ["light sea green", "#20B2AA"],
    ["light sky blue", "#87CEFA"],
    ["light slate gray", "#778899"],
    ["light steel blue", "#B0C4DE"],
    ["light yellow", "#FFFFE0"],
    ["Lime", "#00FF00"],
    ["lime", "#00FF00"],
    ["lime green", "#32CD32"],
    ["linen", "#FAF0E6"],
    ["Magenta-Fuchsia", "#FF00FF"],
    ["Maroon", "#800000"],
    ["maroon", "#800000"],
    ["medium aqua marine", "#66CDAA"],
    ["medium blue", "#0000CD"],
    ["medium orchid", "#BA55D3"],
    ["medium purple", "#9370DB"],
    ["medium sea green", "#3CB371"],
    ["medium slate blue", "#7B68EE"],
    ["medium spring green", "#00FA9A"],
    ["medium turquoise", "#48D1CC"],
    ["medium violet red", "#C71585"],
    ["midnight blue", "#191970"],
    ["mint cream", "#F5FFFA"],
    ["misty rose", "#FFE4E1"],
    ["moccasin", "#FFE4B5"],
    ["navajo white", "#FFDEAD"],
    ["Navy", "#000080"],
    ["navy", "#000080"],
    ["old lace", "#FDF5E6"],
    ["Olive", "#808000"],
    ["olive", "#808000"],
    ["olive drab", "#6B8E23"],
    ["orange", "#FFA500"],
    ["orange red", "#FF4500"],
    ["orchid", "#DA70D6"],
    ["pale golden rod", "#EEE8AA"],
    ["pale green", "#98FB98"],
    ["pale turquoise", "#AFEEEE"],
    ["pale violet red", "#DB7093"],
    ["papaya whip", "#FFEFD5"],
    ["peach puff", "#FFDAB9"],
    ["peru", "#CD853F"],
    ["pink", "#FFC0CB"],
    ["plum", "#DDA0DD"],
    ["powder blue", "#B0E0E6"],
    ["Purple", "#800080"],
    ["purple", "#800080"],
    ["Red", "#FF0000"],
    ["red", "#FF0000"],
    ["rosy brown", "#BC8F8F"],
    ["royal blue", "#4169E1"],
    ["saddle brown", "#8B4513"],
    ["salmon", "#FA8072"],
    ["sandy brown", "#F4A460"],
    ["sea green", "#2E8B57"],
    ["sea shell", "#FFF5EE"],
    ["sienna", "#A0522D"],
    ["Silver", "#C0C0C0"],
    ["silver", "#C0C0C0"],
    ["sky blue", "#87CEEB"],
    ["slate blue", "#6A5ACD"],
    ["slate gray", "#708090"],
    ["snow", "#FFFAFA"],
    ["spring green", "#00FF7F"],
    ["steel blue", "#4682B4"],
    ["tan", "#D2B48C"],
    ["Teal", "#008080"],
    ["teal", "#008080"],
    ["thistle", "#D8BFD8"],
    ["tomato", "#FF6347"],
    ["turquoise", "#40E0D0"],
    ["violet", "#EE82EE"],
    ["wheat", "#F5DEB3"],
    ["White", "#FFFFFF"],
    ["white", "#FFFFFF"],
    ["white smoke", "#F5F5F5"],
    ["Yellow", "#FFFF00"],
    ["yellow", "#FFFF00"],
    ["yellow green", "#9ACD32"]
])

In [6]:
colors

{'Black': '#000000',
 'Blue': '#0000FF',
 'Cyan-Aqua': '#00FFFF',
 'Gray': '#808080',
 'Green': '#008000',
 'Lime': '#00FF00',
 'Magenta-Fuchsia': '#FF00FF',
 'Maroon': '#800000',
 'Navy': '#000080',
 'Olive': '#808000',
 'Purple': '#800080',
 'Red': '#FF0000',
 'Silver': '#C0C0C0',
 'Teal': '#008080',
 'White': '#FFFFFF',
 'Yellow': '#FFFF00',
 'alice blue': '#F0F8FF',
 'antique white': '#FAEBD7',
 'aqua': '#00FFFF',
 'aqua marine': '#7FFFD4',
 'azure': '#F0FFFF',
 'beige': '#F5F5DC',
 'bisque': '#FFE4C4',
 'black': '#000000',
 'blanched almond': '#FFEBCD',
 'blue': '#0000FF',
 'blue violet': '#8A2BE2',
 'brown': '#A52A2A',
 'burly wood': '#DEB887',
 'cadet blue': '#5F9EA0',
 'chart reuse': '#7FFF00',
 'chocolate': '#D2691E',
 'coral': '#FF7F50',
 'corn flower blue': '#6495ED',
 'corn silk': '#FFF8DC',
 'crimson': '#DC143C',
 'cyan': '#00FFFF',
 'dark blue': '#00008B',
 'dark cyan': '#008B8B',
 'dark golden rod': '#B8860B',
 'dark gray-dark grey': '#A9A9A9',
 'dark green': '#006400',


In [12]:
strands_dict = {
"GR_amp_s" : [inp["p_red"][0],xtr["bc_r1"],xtr["bc_ri"],xtr["bc_r2"],inp["GR_OE"][0] ] ,
"GR_OEL_s" : [sq.prime(inp["GR_overlap"][0]),sq.prime(xtr["GR_evL"]),sq.prime(inp["GR_OE"][0])],
"GR_OET_s" : [inp["Tar_overlap"][0],sq.prime(xtr["GR_evT"]),sq.prime(inp["GR_OE"][0])],
"GR_monL_s" : [inp["p_red"][0],xtr["bc_r1"],xtr["bc_ri"],xtr["bc_r2"],inp["GR_OE"][0],xtr["GR_evL"],inp["GR_overlap"][0]],
"GR_monT_s" : [inp["p_red"][0],xtr["bc_r1"],xtr["bc_ri"],xtr["bc_r2"],inp["GR_OE"][0],xtr["GR_evT"],sq.prime(inp["Tar_overlap"][0])],
"BY_amp_s" : [inp["p_yel"][0],xtr["bc_y1"],xtr["bc_yi"],xtr["bc_y2"],inp["BY_OE"][0]],
"BY_OEL_s" :[inp["GR_overlap"][0],sq.prime(xtr["BY_evL"]),sq.prime(inp["BY_OE"][0])],
"BY_OET_s" : [inp["Tar_overlap"][0],sq.prime(xtr["BY_evT"]),sq.prime(inp["BY_OE"][0])],
"BY_monL_s" :[inp["p_yel"][0],xtr["bc_y1"],xtr["bc_yi"],xtr["bc_y2"],inp["BY_OE"][0],xtr["BY_evL"],sq.prime(inp["GR_overlap"][0])],
"BY_monT_s" :[inp["p_yel"][0],xtr["bc_y1"],xtr["bc_yi"],xtr["bc_y2"],inp["BY_OE"][0],xtr["BY_evT"],sq.prime(inp["Tar_overlap"][0])],
"Tar_mon_s" :  [inp["p_tar"][0],xtr["bc_tar"],inp["Tar_overlap"][0]],
"full_loc_r_s" :  [inp["p_red"][0],xtr["bc_r1"],xtr["bc_ri"],xtr["bc_r2"],inp["GR_OE"][0],xtr["GR_evL"],inp["GR_overlap"][0],sq.prime(xtr["BY_evL"]),sq.prime(inp["BY_OE"][0]),sq.prime(xtr["bc_y2"]),sq.prime(xtr["bc_yi"]),sq.prime(xtr["bc_y1"]),sq.prime(inp["p_yel"][0])],
"full_loc_y_s" :[inp["p_yel"][0],xtr["bc_y1"],xtr["bc_yi"],xtr["bc_y2"],inp["BY_OE"][0],xtr["BY_evL"],sq.prime(inp["GR_overlap"][0]),sq.prime(xtr["GR_evL"]),sq.prime(inp["GR_OE"][0]),sq.prime(xtr["bc_r2"]),sq.prime(xtr["bc_ri"]),sq.prime(xtr["bc_r1"]),sq.prime(inp["p_red"][0])],
"full_tar_r_s" :[inp["p_red"][0],xtr["bc_r1"],xtr["bc_ri"],xtr["bc_r2"],inp["GR_OE"][0],xtr["GR_evT"],sq.prime(inp["Tar_overlap"][0]),sq.prime(xtr["bc_tar"]),sq.prime(inp["p_tar"][0])],
"full_tar_y_s" :[inp["p_yel"][0],xtr["bc_y1"],xtr["bc_yi"],xtr["bc_y2"],inp["BY_OE"][0],xtr["BY_evT"],sq.prime(inp["Tar_overlap"][0]),sq.prime(xtr["bc_tar"]),sq.prime(inp["p_tar"][0])],
"full_tar_tarR_s" :[inp["p_tar"][0],xtr["bc_tar"],inp["Tar_overlap"][0],sq.prime(xtr["GR_evT"]),sq.prime(inp["GR_OE"][0]),sq.prime(xtr["bc_r2"]),sq.prime(xtr["bc_ri"]),sq.prime(xtr["bc_r1"]),sq.prime(inp["p_red"][0])],
"full_tar_tarY_s" :[inp["p_tar"][0],xtr["bc_tar"],inp["Tar_overlap"][0],sq.prime(xtr["BY_evT"]),sq.prime(inp["BY_OE"][0]),sq.prime(xtr["bc_y2"]),sq.prime(xtr["bc_yi"]),sq.prime(xtr["bc_y1"]),sq.prime(inp["p_yel"][0])],
    }

In [14]:
dwg = svgwrite.Drawing('test.svg', profile='tiny')
for strand in strands_dict.keys():
    for dom in strands_dict[strand]:
        dwg.add(dwg.text(dom, insert=(0, 0.2), fill=colors['Maroon']))
dwg.save()        